In [2]:
%pip install duckdb agentops

  Using cached duckdb-1.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (762 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.8 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import duckdb


In [39]:
duckdb.sql("create table netflix as SELECT * FROM read_csv('./data/netflix_titles.csv')")

#output = duckdb.sql("describe netflix")

print(output)


CatalogException: Catalog Error: Table with name "netflix" already exists!

In [ ]:
%pip install crewai[tools]


Note: you may need to restart the kernel to use updated packages.


In [8]:


import os
import requests
import warnings
import io
import pandas as pd
from crewai import Agent, Task, Crew

warnings.filterwarnings('ignore')

In [37]:
def get_netflix_data(query):
    data = duckdb.sql(query)
    return data 

print(get_netflix_data('describe netflix'))

duckdb.sql('describe netflix')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ show_id      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ director     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ cast         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ country      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date_added   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ release_year │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ rating       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ duration     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ list

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ show_id      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ director     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ cast         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ country      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date_added   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ release_year │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ rating       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ duration     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ list

### Setting up OpenAI

In [22]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [23]:
os.environ["OPENAI_MODEL_NAME"]='gpt-4o-mini'

### Creating documentation

In [24]:
from crewai_tools import BaseTool

class DatabaseQuery(BaseTool):
    name: str = "Database Query"
    description: str = "Returns the result of SQL query execution"

    def _run(self, sql_query: str) -> str:
        # Implementation goes here
        return get_netflix_data(sql_query)
database_query_tool = DatabaseQuery()

In [25]:
class TableStructure(BaseTool):
    name: str = "Table structure"
    description: str = "Returns the list of columns and their types"

    def _run(self, table: str) -> str:
        table = table.strip('"').strip("'")
        return get_netflix_data('describe {table}')

class TableExamples(BaseTool):
    name: str = "Table examples"
    description: str = "Returns the first N rows from the table"

    def _run(self, table: str, n: int = 30) -> str:
        table = table.strip('"').strip("'")
        return get_netflix_data('select * from {table} limit {n}')

table_structure_tool = TableStructure()
table_examples_tool = TableExamples()

In [26]:
database_specialist_agent = Agent(
    role = "database specialist",
    goal = "Provide data to answer business questions using SQL",
    backstory = '''
    You are an expert in SQL, so you can help the team to gather needed data to power their decisions.
    You are very accurate and take into account all the nuances in data.''',
    allow_delegation=False,
    verbose=True
)

In [27]:
tech_writer_agent = Agent(
    role = "technical writer",
    goal = "Write engaging and factually accurate technical documentation for data sources or tools",
    backstory = '''
    You are an expert in both technology and communications, so you can easily explain even sophisticated concepts.
    Your texts are concise and can be easily understood by wide audience.
    You use professional but rather informal style in your communication.
    You base your work on the factual information provided by your colleagues.
    You stick to the facts in the documentation and use ONLY information provided by the colleagues not adding anything.
    ''',
    allow_delegation=False,
    verbose=True
)

In [28]:
table_description_task = Task(
    description = '''Provide the comprehensive overview for the data in table {table}, so that
    it's easy to understand the structure of the data. This task is crucial to put together the documentation for our database''',
    expected_output = '''The comprehensive overview of {table} in the md format. Include 2 sections: columns (list of columns with their types) and examples (the first 10 rows from table).''',
    tools = [database_query_tool, table_structure_tool, table_examples_tool],
    agent = database_specialist_agent
)

In [29]:
table_documentation_task = Task(
    description = '''Using provided information about the table, put together the detailed documentation for this table so that people can use it in practice''',
    expected_output = '''Well-written detailed documentation describing the data scheme for the table {table} in markdown format,
    that gives the table overview in 1-2 sentences then then describes each columm. Structure the columns description as a markdown table with column name, type and description.
    The documentation is based ONLY on the information provided by the database specialist without any additions.
    ''',
    tools = [],
    output_file="table_documentation.md",
    agent = tech_writer_agent
)

In [30]:
qa_specialist_agent = Agent(
    role = "Quality Assurance specialist",
    goal = "Ensure the highest quality of the documentation we provide (that it's correct and easy to understand)",
    backstory = '''
    You work as a Quality Assurance specialist, checking the work from the technical writer and ensuring that it's inline with our highest standards.
    You need to check that the technical writer provides the full complete answers and make no assumptions.
    Also, you need to make sure that the documentation addresses all the questions and is easy to understand.
    ''',
    allow_delegation=True,
    verbose=True
)

In [31]:
qa_review_task = Task(
    description = '''
    Review the draft documentation provided by the technical writer.
    Ensure that the documentation fully answers all the questions: the purpose of the table and its structure in the form of markdown table.
    Make sure that the documentation is consistent with the  information provided by the database specialist.
    Double check that there are no irrelevant comments in the final version of documentation.
    ''',
    expected_output = '''
    The final version of the documentation in markdown format that can be published.
    The documentation should fully address all the questions, be consistent and follow our professional but informal tone of voice.
    ''',
    tools = [],
    context = [table_description_task, table_documentation_task],
    output_file="checked_table_documentation.md",
    agent = qa_specialist_agent
)

In [33]:
full_crew = Crew(
    agents=[database_specialist_agent, tech_writer_agent, qa_specialist_agent],
    tasks=[table_description_task,  table_documentation_task, qa_review_task],
    verbose=True,
    memory = True
)

In [ ]:
#%%time

full_result = full_crew.kickoff({'table': 'netflix'})

In [ ]:
from IPython.display import Markdown
Markdown(full_result.replace("```", ""))

In [ ]:
#%%time
full_sessions_result = full_crew.kickoff({'table': 'ecommerce_db.sessions'})

In [ ]:
from IPython.display import Markdown
Markdown(full_sessions_result.replace("```", ""))